### Importieren der notwendigen Klassen

In [ ]:
from tkinter import *
from tkinter import ttk
from tkinter.messagebox import showinfo
from tkinter import scrolledtext
from tkinter import filedialog

### Konstanten definieren

In [ ]:
NochImSpiel = 0
KI_gewonnen = 1
KI_verloren = 2

name = {True:"Bob, die KI", False:"Alice"}

### Die zentrale Klasse

In [ ]:
class muenzen():
    
    def __init__(self, dim):
        self.dim = dim # wieviele Münzen liegen beim Start auf dem Tisch?
        self.muenzenAufTisch = dim
        self.sinnvolleZuege = {}
        self.letzterGemerkterIndex = 0
        self.memory = {}
        for i in range(1, self. dim + 1):
            self.sinnvolleZuege[i] = [1,2] # mögliche Züge
            self.memory[i] = None # nichts gemerkt
        self.sinnvolleZuege[1] = [1] # mögliche Züge  bei einer Münze
        self.ende_status = NochImSpiel # das Spiel läuft noch
        self.begin = False # False = der Gegner; True = die KI
        self.am_zug = self.begin
        self.comment = ""

    # Ein neuer Durchgang, falls einer der beiden verloren hat
    # Der jeweils andere Spieler beginnt dann
    def reset(self):
        if self.ende_status == NochImSpiel:
            self.comment = "Spiel noch nicht beendet"
            return

        self.muenzenAufTisch = self.dim
        if self.ende_status == KI_gewonnen:
            self.KI_memory_back()
        elif self.ende_status == KI_verloren:
            for i in range(1, self. dim + 1):
                self.memory[i] = None
            self.letzterGemerkterIndex = 0
            
        self.ende_status = NochImSpiel
        self.begin = not self.begin # jetzt fängt der jeweils andere an
        self.am_zug = self.begin
        self.comment = "Reset erfolgt!"
        
    def KI_merken(self, muenzen, zug):
        self.comment = ""
        if zug in self.sinnvolleZuege[muenzen]:
            self.sinnvolleZuege[muenzen].remove(zug)
            self.memory[muenzen] = zug
            self.letzterGemerkterIndex = muenzen
            self.comment = f"gemerkt von Zug {zug} bei {muenzen} Münzen!"

        else:
            self.comment = f"KI_merken von Zug {zug} bei {muenzen} Münzen nicht erlaubt!"
            
    def KI_memory_back(self):
        mem = self.memory[self.letzterGemerkterIndex]
        if mem != None:
            self.memory[self.letzterGemerkterIndex] = None
            self.sinnvolleZuege[self.letzterGemerkterIndex].append(mem)
            self.letzterGemerkterIndex = 0

    def sinnvolle_KI_zuege(self, anzahl_muenzen):
        zuege = []
        for zug in self.sinnvolleZuege[anzahl_muenzen]:
            zuege.append(zug)
        return zuege
    
    def moegliche_KI_zuege(self, anzahl_muenzen):
        if self.sinnvolleZuege[anzahl_muenzen] == []:
            if anzahl_muenzen == 1:
                return [1]
            else:
                return [1,2]
        else:
            return self.sinnvolle_KI_zuege(anzahl_muenzen)
    
    def moegliche_human_zuege(self, anzahl_muenzen):
        if anzahl_muenzen == 1:
            return [1]
        else:
            return [1,2]
                
    def wegnehmen(self, anzahl): # Alice nimmt
        self.comment = ""
        if self.am_zug:
            self.comment = f"{name[False]} ist nicht am Zug!"
            return
        
        if (self.muenzenAufTisch == 1 and anzahl > 1) or anzahl > 2:
            self.comment = f"Wegnehmen von {anzahl} Münzen nicht erlaubt!"
            return
            
        self.comment = f"{name[False]} hat {anzahl} Münzen genommen."
        self.muenzenAufTisch -= anzahl
            
        
        if self.muenzenAufTisch == 0: # Alice hat verloren
            self.ende_status = KI_gewonnen
        else:
            self.am_zug = True # jetzt ist KI dran
            
    def KI_move(self, anzahl):
        if not self.am_zug:
            self.comment = "KI ist nicht am Zug!"
            return
        
        if anzahl > 2 or (self.muenzenAufTisch == 1 and anzahl > 1):
            self.comment = f"KI kann nicht {anzahl} Münzen nehmen!"
               
        if self.sinnvolleZuege[self.muenzenAufTisch] == []: 
            self.memory[self.letzterGemerkterIndex] = None
            self.letzterGemerkterIndex = 0
            self.muenzenAufTisch -= anzahl
            self.comment = f"Kein Sinnvoller Zug mehr möglich. {name[True]} hat {anzahl} Münzen genommen."

            if self.muenzenAufTisch == 0: # KI hat verloren
                self.ende_status = KI_verloren
            else:
                self.am_zug = False # jetzt ist Alice dran
        else:   
            if anzahl in self.sinnvolle_KI_zuege(self.muenzenAufTisch):
                if self.letzterGemerkterIndex != 0:
                    self.KI_memory_back()
                    
                self.KI_merken(self.muenzenAufTisch, anzahl)
                self.muenzenAufTisch -= anzahl
                self.comment = f"{name[True]} hat {anzahl} Münzen genommen. Ist ggf. sinnvoll?."
                  
                if self.muenzenAufTisch == 0: # Ich habe verloren
                    self.ende_status = KI_verloren
                else:
                    self.am_zug = False # jetzt ist Alice dran
            else:
                self.comment = f"Zug {anzahl} nicht erlaubt!"            
            
    def spiel_status_text(self):
        if self.ende_status == NochImSpiel:
            return f"Am Zug ist {name[self.am_zug]}"
        elif self.ende_status == KI_gewonnen:
            return name[True] + " hat gewonnen!"
        else:
            return name[True] + " hat verloren!"
        
    def out_board(self):
        out = ""
        if self.muenzenAufTisch == 0:
            out += (f"Es ist keine Münze mehr auf dem Tisch\n\n")
        elif self.muenzenAufTisch == 1:
            out +=(f"Es ist {self.muenzenAufTisch} Münze auf dem Tisch\n\n")
        else:
            out +=(f"Es sind {self.muenzenAufTisch} Münzen auf dem Tisch\n\n")
        
        for i in range(self.dim,0,-1):
            if self.memory[i] == None:
                past = ""
            else:
                past = "Memory " + str(self.memory[i])
            if i == self.muenzenAufTisch:
                pre = "> "
            else:
                pre = "   "
            out +=(f"{pre}Status: {i} Münze(n)\t\t sinnvolle Züge: {self.sinnvolleZuege[i]}\t\t {past}\n")
        
        out += m.spiel_status_text()

        if self.muenzenAufTisch != 0:
            mz = "machbare"
            if self.am_zug:
                zuege = self.moegliche_KI_zuege(self.muenzenAufTisch)
                if self.sinnvolleZuege[self.muenzenAufTisch] != []:
                    mz = "sinnvolle"
            else:
                zuege = self.moegliche_human_zuege(self.muenzenAufTisch)
                            
            out +=(f"\nFür {name[self.am_zug]} {mz} Züge: {zuege}\n")
        
        return out
            
    def print_board(self):
        print("Comment: " + self.comment)
        print(self.out_board())
        print("--------")
        
    

### Die GUI

In [ ]:
m = muenzen(6)

def reset_action():
    m.reset()
    reset_button.config(state="disabled")
    zug_eingabebox.config(state='readonly')

    out_state = m.out_board()
    text_area.config(state = "normal")
    text_area.delete('0.0', END)
    text_area.insert(INSERT,out_state + "\n")
    text_area.config(state = "disabled")

    out = m.comment
    comment_area.config(state = "normal")
    comment_area.delete('0.0', END)
    comment_area.insert(INSERT,out)
    comment_area.config(state = "disabled")
    
    labText = f"Wieviele Münzen nimmt {name[m.am_zug]}?"
    labelZug.config(text = labText)
   
    if m.ende_status == NochImSpiel:  # das sollte doch so sein!!!
        if m.am_zug:
            allowed = m.moegliche_KI_zuege(m.muenzenAufTisch)
        else:
            allowed = m.moegliche_human_zuege(m.muenzenAufTisch)
        zug_eingabebox['values'] = allowed
        zug_eingabebox.set('')
    else:
        zug_eingabebox.set('')
    
def act(event):
    zahltext = selected_Zug.get()
    #text_area.delete('0.0', END)
    
    if not m.am_zug:
        m.wegnehmen(int(zahltext))
    else:
        m.KI_move(int(zahltext))
    
    out_state = m.out_board()
    text_area.config(state = "normal")
    text_area.delete('0.0', END)
    text_area.insert(INSERT,out_state)
    text_area.config(state = "disabled")

    out = m.comment
    if out == "":
        out = "No Comment\n"
    comment_area.config(state = "normal")
#    comment_area.delete('0.0', END)
    comment_area.insert(INSERT,out + '\n')
    comment_area.config(state = "disabled")
    
    labText = f"Wieviele Münzen nimmt {name[m.am_zug]}?"
    labelZug.config(text = labText)
    
    if m.ende_status == NochImSpiel:
        if m.am_zug:
            allowed = m.moegliche_KI_zuege(m.muenzenAufTisch)
        else:
            allowed = m.moegliche_human_zuege(m.muenzenAufTisch)
        zug_eingabebox['values'] = allowed
        zug_eingabebox.set('')
    else:
        reset_button.config(state="normal")
        zug_eingabebox.set('')
        zug_eingabebox.config(state = "disabled")
        
# MainWindow ---------------------

fenster = Tk()
fenster.title("Münzspiel Alice gegen Bob, die KI")
fenster.resizable(width=False, height=False)

reset_button = Button(fenster, 
                      text="neue Runde", 
                      command=reset_action, 
                      state='disabled',                     
                      relief = "raised", # geht in MACOS nicht!!!
                     )
                    
reset_button.pack()

# InputWindow ---------------------

eingabe = Frame(fenster, relief = RIDGE)
eingabe.pack(side = TOP)

labelZug = Label(eingabe)
labText = f"Wieviele Münzen nimmt {name[m.am_zug]}?"
    
labelZug.config(text = labText, borderwidth=2, relief="ridge")
labelZug.pack(padx=5, pady=5, side = TOP)

selected_Zug = StringVar()
zug_eingabebox = ttk.Combobox(eingabe, 
                      textvariable=selected_Zug,
                     )

zug_eingabebox['state'] = 'readonly'

if m.am_zug:
    allowed = m.moegliche_KI_zuege(m.muenzenAufTisch)
else:
    allowed = m.moegliche_human_zuege(m.muenzenAufTisch)
    
zug_eingabebox['values'] = allowed
zug_eingabebox.set('')
zug_eingabebox.bind('<<ComboboxSelected>>', act)
zug_eingabebox.pack(padx=5, pady=5, side = RIGHT)

# TextausgabeWindow ---------------------

text_ausgabe = Frame(fenster)
text_ausgabe.pack(padx = 5, pady = 5)

slider_textfield = Scrollbar(text_ausgabe, orient = VERTICAL)

text_area = Text(text_ausgabe, 
                wrap = WORD, 
                width = 50, 
                height = 10, 
                font = ("Times New Roman", 15),
                state='normal',
                yscrollcommand = slider_textfield.set,
                relief = 'sunken',
                borderwidth = 2
                )
  
out_state = m.out_board()
text_area.insert(INSERT,out_state)
text_area.config(state = "disabled")
slider_textfield.config(command = text_area.yview)

slider_textfield.pack(side = RIGHT, fill = Y)

text_area.pack(pady = 5, padx = 5)

# CommentWindow ---------------------

comment_ausgabe = Frame(fenster)
comment_ausgabe.pack(padx = 5, pady = 5)

slider_commentfield = Scrollbar(comment_ausgabe, orient = VERTICAL)

comment_area = Text(comment_ausgabe,
                wrap = WORD, 
                width = 50, 
                height = 10, 
                font = ("Times New Roman", 15),
                state='normal',
                yscrollcommand = slider_commentfield.set,
                relief = 'sunken',
                borderwidth = 2,
                )
comment_area.insert(INSERT, "NoComment\n")
comment_area.config(state = "disabled")
slider_commentfield.config(command = comment_area.yview)

slider_commentfield.pack(side = RIGHT, fill = Y)

comment_area.pack(padx = 5, pady = 5)

# Main

fenster.mainloop()